## Building Agentic Workflows with Model Context Protocol on Azure

✍️ AI Applications are evolving rapidly. With every new AI application built, orgnaziations are facing the the complexity of managing AI Assistants, Tools and Data sources increase. As the AI applications evolve, organizations want to add more tools and embed the new knowledge or intelligence into existing applictions. However, the challenge always is that there is no standard protocol to connect LLMs to AI Assistants, tools and datasources. 


Every new 

In [ ]:
## Install dependencies
# %pip install -q mcp[cli] azure-identity python-dotenv azure-ai-projects aiohttp

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.112.1 requires starlette<0.39.0,>=0.37.2, but you have starlette 0.46.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Download the dataset from the URL and save it to ./data folder

```
#!/bin/bash
curl -L -o ~/Downloads/smartphones.zip\
  https://www.kaggle.com/api/v1/datasets/download/amansingh0000000/smartphones
```


### Deploy Agents to Azure AI Agentic Service

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FileSearchTool, MessageAttachment, FilePurpose
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

project_client = AIProjectClient.from_connection_string(credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"])
    
# We will upload the local file and will use it for vector store creation.
#upload a file
file = project_client.agents.upload_file_and_poll(file_path='../data/research_paper.pdf', purpose=FilePurpose.AGENTS)
print(f"Uploaded file, file ID: {file.id}")

# create a vector store with the file you uploaded
vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

# create a file search tool
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

# notices that FileSearchTool as tool and tool_resources must be added or the agent will be unable to search the file
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="research-assistant-agent",
    instructions="You are a helpful research assistant. You will answer questions based on the research paper.",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)     

Uploaded file, file ID: assistant-FPCRh6h6vWH254T62x85WJ
Created vector store, vector store ID: vs_UQHorx2KCyBEPX2HoEfnwwMo


### Connect to Agents using Claude Desktop Client

### Connect to Agents using our own console client